In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from Bio import Entrez
from tqdm import tqdm
import pickle

In [21]:
data = pd.read_csv (r'data/SFARI-Genes.csv')
cols = data.columns
N, D = data.shape
print(cols)

Index(['status', 'gene-symbol', 'gene-name', 'ensembl-id', 'chromosome',
       'genetic-category', 'gene-score', 'syndromic', 'number-of-reports'],
      dtype='object')


In [29]:
def save_to_txt_file(contents, filename):
    DIR = os.getcwd() + "/results/" + filename
    
    with open(DIR, "w") as f:
        f.write(contents)
        
def save_to_pickle_file(contents, filename):
    DIR = os.getcwd() + "/data/" + filename
    
    with open(DIR, "wb") as handle:
        pickle.dump(contents, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def open_pickle_file(filename):
    DIR = os.getcwd() + "/data/" + filename
    
    with open(DIR, "rb") as handle:
        data = pickle.load(handle)
        
    return data

# Task 1

In [23]:
gene_symbols = data['gene-symbol'].unique()
print(gene_symbols.size)

1023


In [5]:
# Save Entrez scores

In [24]:
Entrez.email = "s1803764@ed.ac.uk"
output = ""
eid = {}

for i in tqdm(range(gene_symbols.size)):
    g = gene_symbols[i]
    handle = Entrez.esearch(db="gene", retmax=10, term=g + "[sym] homo sapiens[Organism]")
    record = Entrez.read(handle)
    #output += g + ": " + ", ".join(str(x) for x in record['IdList'])
    #output += "\n"
    eid[g] = record['IdList']
    
#save_to_txt_file(output, "task2-1-eid.txt")

  1%|▎                                         | 7/1023 [00:04<11:36,  1.46it/s]


KeyboardInterrupt: 

In [22]:
save_to_pickle_file(eid, "eid.pickle")

# Task 2

In [6]:
gene2go = pd.read_csv(os.getcwd() + "/data/gene2go", dtype=str, delimiter="\t")
save_to_pickle_file(gene2go, "gene2go.pickle")

In [25]:
gene2go.shape

(2084178, 8)

In [30]:
gene2go = open_pickle_file("gene2go.pickle")
eid = open_pickle_file("eid.pickle")

TypeError: __new__() missing 2 required positional arguments: 'tag' and 'attributes'

In [19]:
go_terms = {}

tax_ids = gene2go["#tax_id"]
human_gene_indexes = tax_ids[tax_ids == "9606"]
human_genes = gene2go.filter(items= human_gene_indexes, axis = 0)

for g in tqdm(gene_symbols):
    gene_entrez = eid[g][0]

    human_gene_ids = human_genes["GeneID"]
    gene_index = human_gene_ids[human_gene_ids == gene_entrez].unique()

    if gene_index.size >= 1:
        g_index = int(gene_index[0])


    go_term = gene2go.iloc[g_index]["GO_term"]
    go_terms[g] = go_term

    print(g)
    print(gene_entrez)
    print(go_term)
    print()

save_to_pickle_file(go_terms, "go-terms.pickle")

  0%|                                                  | 0/1023 [00:00<?, ?it/s]


NameError: name 'eid' is not defined

In [18]:
go_terms

{}